In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import json

import llama_cpp
import torch

from curverag import utils
from curverag.curverag import CurveRAG, DEFAULT_ENTITY_TYPES, DEFAULT_GLINER_MODEL
from curverag.graph import KnowledgeGraph
from curverag.atth.kg_dataset import KGDataset
from curverag.atth.models.hyperbolic import AttH

In [6]:
import os
from dotenv import load_dotenv
from openai import OpenAI
load_dotenv() 

True

## Dataset and LLM (OpenAI)

In [7]:
docs = [
    "The patient was diagnosed with type 2 diabetes mellitus and prescribed metformin 500mg twice daily.",
    "MRI scan revealed a small lesion in the left temporal lobe suggestive of low-grade glioma.",
    "Administer 5mg of lorazepam intravenously for acute seizure management.",
    "Blood tests showed elevated ALT and AST levels, indicating possible liver inflammation.",
    "The subject reported chronic lower back pain, managed with physical therapy and NSAIDs.",
    "CT angiography confirmed the presence of a pulmonary embolism in the right lower lobe.",
    "The patient underwent coronary artery bypass graft surgery without complications.",
    "Routine vaccination included MMR, tetanus, and influenza immunizations.",
    "Histopathology indicated ductal carcinoma in situ (DCIS) in the breast biopsy sample.",
    "The child presented with a persistent cough and fever, diagnosed as streptococcal pharyngitis."
]


In [9]:
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

## Fit CurveRAG

In [12]:
entity_types = ['people', 'locations', 'entities', 'movies', 'directors']
rag = CurveRAG(
    openai_client=client,
    entity_types=entity_types,
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [13]:
rag.fit(docs=docs, dataset_name='test_run')

creating graph


 10%|█████████████████▌                                                                                                                                                              | 1/10 [00:03<00:29,  3.30s/it]

upserting


 20%|███████████████████████████████████▏                                                                                                                                            | 2/10 [00:05<00:22,  2.76s/it]

upserting


...

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:41<00:00,  4.15s/it]
2025-06-14 00:27:28,700 INFO     Saving logs in: ./logs/06_14/test_run/AttH_00_27_28
2025-06-14 00:27:28,701 INFO     	 (19, 14, 19)
2025-06-14 00:27:28,710 INFO     Total number of parameters 89052
2025-06-14 00:27:28,713 INFO     	 Start training


upserting
num nodes 19
unique node ids {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}
creating dataset
train kg embeddings
max_epochs 50


train loss: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 26/26 [00:00<00:00, 161.27ex/s, loss=0.6883]
2025-06-14 00:27:28,877 INFO     	 Epoch 0 | average train loss: 0.6883
2025-06-14 00:27:28,882 INFO     	 Epoch 0 | average valid loss: 2.0525
train loss: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 26/26 [00:00<00:00, 221.64ex/s, loss=2.1058]
2025-06-14 00:27:29,000 INFO     	 Epoch 1 | average train loss: 2.1058
2025-06-14 00:27:29,004 INFO     	 Epoch 1 | average valid loss: 1.9333
train loss: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 26/26 [00:00<00:00, 220.28ex/s, loss=1.9965]
2025-06-14 00:27:29,124 INFO     	 Epoch 2 | average train loss: 1.9965
2

returning model


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [14]:
len(rag.graph.nodes)

19

In [16]:
rag.graph.nodes

[Node(id=1, name='Type 2 Diabetes Mellitus', description='A chronic condition that affects the way the body processes blood sugar. A chronic condition that affects the way the body processes blood sugar.', alias=['Diabetes Mellitus Type 2', 'Type 2 Diabetes Mellitus'], additional_information=['Chronic metabolic disorder', 'Requires management and treatment']),
 Node(id=2, name='Metformin', description='A medication used to treat type 2 diabetes. A medication used to treat type 2 diabetes.', alias=['Glucophage', 'Metformin'], additional_information=['500mg dosage', 'Taken twice daily']),
 Node(id=3, name='Low-Grade Glioma', description='A type of brain tumor that is typically slow growing. A type of brain tumor that is typically slow growing.', alias=['Low Grade Glioma', 'Low-Grade Glioma'], additional_information=['Can affect brain function', 'Requires monitoring and potential treatment']),
 Node(id=4, name='Temporal Lobe', description='Region of the brain associated with processing au

In [22]:
rag.graph.nodes

[Node(id=1, name='Type 2 Diabetes Mellitus', description='A chronic condition that affects the way the body processes blood sugar. A chronic condition that affects the way the body processes blood sugar.', alias=['Diabetes Mellitus Type 2', 'Type 2 Diabetes Mellitus'], additional_information=['Chronic metabolic disorder', 'Requires management and treatment']),
 Node(id=2, name='Metformin', description='A medication used to treat type 2 diabetes. A medication used to treat type 2 diabetes.', alias=['Glucophage', 'Metformin'], additional_information=['500mg dosage', 'Taken twice daily']),
 Node(id=3, name='Low-Grade Glioma', description='A type of brain tumor that is typically slow growing. A type of brain tumor that is typically slow growing.', alias=['Low Grade Glioma', 'Low-Grade Glioma'], additional_information=['Can affect brain function', 'Requires monitoring and potential treatment']),
 Node(id=4, name='Temporal Lobe', description='Region of the brain associated with processing au

In [21]:
rag.graph.edges

[Edge(source=1, target=2, name='Prescribed', is_directed=True, description='Medication prescribed for diabetes', notes=['Patient management', 'Diabetes treatment']),
 Edge(source=3, target=4, name='Located In', is_directed=False, description='Presence of tumor in the brain region', notes=['Diagnostic imaging', 'Brain health assessment']),
 Edge(source=5, target=1, name='Used For', is_directed=True, description='Medication used for managing diabetes symptoms', notes=['Seizure control', 'Neuroadverse effects']),
 Edge(source=6, target=8, name='Indicates', is_directed=True, description='Elevated ALT levels indicate liver inflammation', notes=['Liver function tests', 'Clinical assessment']),
 Edge(source=7, target=8, name='Indicates', is_directed=True, description='Elevated AST levels indicate liver inflammation', notes=['Liver function tests', 'Clinical assessment']),
 Edge(source=9, target=10, name='Managed With', is_directed=True, description='Treatment approach for chronic back pain', 

In [29]:
query = "What is Chronic Lower Back Pain?"
rag.query(query, traversal = 'pp')

entities []


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

graph nodes retrieved ['Chronic Lower Back Pain']
similar_node_ids [9, 18, 1, 10, 11]
similar_node_ids graph nodes retrieved ['Type 2 Diabetes Mellitus', 'Chronic Lower Back Pain', 'Physical Therapy', 'NSAIDs', 'Ductal Carcinoma In Situ']
***************************************!!******************************
***************************************!!******************************
***************************************!!******************************
***************************************!!******************************
***************************************!!******************************

You are a helpful assistant analyzing the given input data to provide an helpful response to the user query.

# USER QUERY
What is Chronic Lower Back Pain?

# Context:
KnowledgeGraph Overview
  There are 5 entities and 4 relationships in this graph.

Entities in the graph:
  • 'Type 2 Diabetes Mellitus':
      The entity has the following description: A chronic condition that affects the way the b

2025-06-14 01:39:29,818 INFO     HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


'Chronic lower back pain is defined as persistent pain in the lower back area. It is a common condition that is often a significant source of disability, affecting daily activities and quality of life. Chronic lower back pain can be managed through various approaches, including physical therapy and the use of medications such as nonsteroidal anti-inflammatory drugs (NSAIDs) to relieve pain and reduce inflammation. The treatment involving physical therapy aims to improve mobility and reduce pain using specific exercises and modalities [1][2][3].'